<a href="https://colab.research.google.com/github/JaeHeee/Tensorflow_practice/blob/master/10.mnist_batchnorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.2.0


In [0]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data


In [0]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

In [0]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate) :
    return tf.keras.layers.Dropout(rate)
    
def batch_norm() :
    return tf.keras.layers.BatchNormalization()

In [0]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [0]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [0]:
network = create_model_class(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
start_time = time()

In [31]:
for epoch in range(0, training_epochs):
    for idx, (train_input, train_label) in enumerate(train_dataset):            
        grads = grad(network, train_input, train_label)
        optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))

        train_loss = loss_fn(network, train_input, train_label)
        train_accuracy = accuracy_fn(network, train_input, train_label)
                
        for test_input, test_label in test_dataset:                
            test_accuracy = accuracy_fn(network, test_input, test_label)

        print("Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
              % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
              test_accuracy))

Epoch: [ 0] [    0/  468] time: 1.8673, train_loss: 0.41946381, train_accuracy: 0.3438, test_Accuracy: 0.2273
Epoch: [ 0] [    1/  468] time: 2.4271, train_loss: 0.42746776, train_accuracy: 0.4297, test_Accuracy: 0.3599
Epoch: [ 0] [    2/  468] time: 3.0079, train_loss: 0.30735481, train_accuracy: 0.4922, test_Accuracy: 0.3883
Epoch: [ 0] [    3/  468] time: 3.5851, train_loss: 0.28574267, train_accuracy: 0.4219, test_Accuracy: 0.4125
Epoch: [ 0] [    4/  468] time: 4.1611, train_loss: 0.30497819, train_accuracy: 0.4531, test_Accuracy: 0.4595
Epoch: [ 0] [    5/  468] time: 4.7600, train_loss: 0.33355433, train_accuracy: 0.5156, test_Accuracy: 0.5028
Epoch: [ 0] [    6/  468] time: 5.3354, train_loss: 0.37071297, train_accuracy: 0.5547, test_Accuracy: 0.5384
Epoch: [ 0] [    7/  468] time: 5.9119, train_loss: 0.25107032, train_accuracy: 0.5859, test_Accuracy: 0.5695
Epoch: [ 0] [    8/  468] time: 6.4836, train_loss: 0.32595611, train_accuracy: 0.6094, test_Accuracy: 0.6064
Epoch: [ 0